In [1]:
import pandas as pd  # Φόρτωση της βιβλιοθήκης Pandas για ανάγνωση/επεξεργασία δεδομένων σε πίνακες(DataFrames).
import openpyxl      #Μηχανή/driver για ανάγνωση/εγγραφή αρχείων Excel (.xlsx) που χρησιμοποιεί η Pandas.

 df = pd.read_csv("thyroid_cancer_risk_data.csv")      #Φόρτωση αρχικού dataset από CSV σε DataFrame

df.to_excel('thyroid_cancer_risk_data.xlsx',index = False) # Εξαγωγή σε Excel χωρίς index-στήλη (καθαρό schema)

print(df)   # Γρήγορος οπτικός έλεγχος δομής/δειγμάτων

df=pd.read_excel('thyroid_cancer_risk_data.xlsx') # Επαναφόρτωση από XLSX 

        Patient_ID  Age  Gender  Country       Ethnicity Family_History  \
0                1   66    Male   Russia       Caucasian             No   
1                2   29    Male  Germany        Hispanic             No   
2                3   86    Male  Nigeria       Caucasian             No   
3                4   75  Female    India           Asian             No   
4                5   35  Female  Germany         African            Yes   
...            ...  ...     ...      ...             ...            ...   
212686      212687   58  Female    India           Asian             No   
212687      212688   89    Male    Japan  Middle Eastern             No   
212688      212689   72  Female  Nigeria        Hispanic             No   
212689      212690   85  Female   Brazil  Middle Eastern             No   
212690      212691   46  Female    Japan  Middle Eastern             No   

       Radiation_Exposure Iodine_Deficiency Smoking Obesity Diabetes  \
0                     Yes  

In [2]:
#Αφαίρεση Μη Πληροφοριακών Πεδίων (Irrelevant Columns)


df = df.drop(columns=["Patient_ID"]) #Διαγραφή της στήλης Patient_ID 

df = df.drop(columns=["Country","Ethnicity"]) #Διαγραφή της στήλης Country & Εthnicity 


In [3]:
#Ελεγχος για ύπαρξη  Διπλότυπων Τιμών (Duplicates)

duplicates = df[df.duplicated()] # Εντοπισμός εντελώς όμοιων γραμμών (across all columns)

print(f"Βρέθηκαν {duplicates.shape[0]} διπλότυπες γραμμές.\n") #Αναφορά πλήθους διπλότυπων γραμμών για τεκμηρίωση ποιότητας

Βρέθηκαν 0 διπλότυπες γραμμές.



In [4]:
#Έλεγχος για ύπαρξη Ελλειπουσών Τιμών (Missing Values)

Dirty_values = ["-", " ", "", "N/A", "na", "NaN", "nan"]  #Όλες οι πιθανές τιμές αναπαραστάσεις του "κενου".

for col in df.columns:                           # Διάσχιση όλων των στηλών.
    count = df[col].isin(Dirty_values).sum()     # Μέτρηση εμφανίσεων placeholder ανά στήλη.
if count > 0:
     print(f"Στήλη '{col}' περιέχει {count} κενές τιμές.")     # Αναφορά στηλών με κενές τιμές.

else:
     print("Δεν υπάρχουν κενές τιμές")     # Επιβεβαίωση απουσίας κενών τιμών.


Δεν υπάρχουν κενές τιμές


In [5]:
#Ελεγχος Ακραίων Τιμών (Outliers)

numeric_cols = df.select_dtypes(include=["number"]).columns  #Επιλογή μόνο των στηλών με αριθμητικά δεδομένα.
 
for col in numeric_cols:            
    Q1 = df[col].quantile(0.25)   #1ο τεταρτημόριο (25ο εκατοστημόριο)
    Q3 = df[col].quantile(0.75)   #3ο τεταρτημόριο (75ο εκατοστημόριο)
    IQR = Q3 - Q1                 # Ενδοτεταρτημοριακό εύρος (IQR)
    lower_bound = Q1 - 1.5 * IQR     # Κατώτερο όριο «μη ακραίων» τιμών
    upper_bound = Q3 + 1.5 * IQR     # Ανώτερο όριο «μη ακραίων» τιμών
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]  # Φιλτράρισμα πιθανών outliers
    count = outliers.shape[0]   # Πλήθος outliers ανά στήλη
    
    if count > 0:
        print(f"Στήλη '{col}': {count} outliers")  # Αναφορά ύπαρξης ακραίων τιμών
    else:
        print(f'Η στήλη {col} δεν έχει outliers')   # Επιβεβαίωση απουσίας outliers για κάθε στήλη.
 

Η στήλη Age δεν έχει outliers
Η στήλη TSH_Level δεν έχει outliers
Η στήλη T3_Level δεν έχει outliers
Η στήλη T4_Level δεν έχει outliers
Η στήλη Nodule_Size δεν έχει outliers


In [6]:
#Έλεγχος Ονοματολογικής Ασυνέπειας & Κατηγορικών τιμών (Consistency)

# Εύρεση κατηγορηματικών στηλών (τύπος object ή string)
categorical_cols = df.select_dtypes(include=["object"]).columns

'''Κατηγορικές στήλες στο dataset θεωρούνται οι παρακάτω με τιμές Yes & No:
Family_History
Radiation_Exposure 
Iodine_Deficiency
Smoking
Obesity
Diabetes

'''
unconsinstency = False       # Αρχικοποίηση μεταβλήτής που δηλώνει αν εντοπίστηκαν ασυνέπειες
for col in categorical_cols:         # Βρόχος σε κάθε κατηγορική στήλη
    unique_vals = df[col].dropna().unique()   # Μοναδικές τιμές αγνοώντας τα NaN (αν υπάρχουν)
    lower_vals = pd.Series(unique_vals).str.lower().unique()   #Ίδιες τιμές μετασχηματισμένες σε πεζά για σύγκριση
    
    if len(unique_vals) != len(lower_vals):      #Αν το πλήθος αλλάζει → ύποπτες παραλλαγές γραφής
        unconsinstency = True
        print(f"Στήλη '{col}' περιέχει πιθανές ασυνεπείς τιμές:")         #Αναφορά στήλης με ασυνέπεια
        print(f"   Πρωτότυπες τιμές: {list(unique_vals)}")                #Εμφάνιση αρχικών μοναδικών τιμών
        print(f"   Κανονικοποιημένες (σε πεζά): {list(lower_vals)}\n")    # Εμφάνιση κανονικοποιημένων τιμών
if unconsinstency is False:          # Αν δεν εντοπίστηκαν ασυνέπειες σε καμία στήλη
    print("Καμία κατηγορική στήλη δεν παρουσιάζει ασυνέπεια")

Καμία κατηγορική στήλη δεν παρουσιάζει ασυνέπεια


In [ ]:
#Μετατροπή κατηγορικών τιμών σε αριθμητικές (Label Encoding ή Dummy encoding)

df["Gender"] = df["Gender"].map({"Male": 1, "Female": 0}) #Κωδικοποίηση του πεδίου Gender ώστε όπου Male = 1 και Female =0

df["Family_History"] = df["Family_History"].map({"Yes": 1, "No": 0}) #Κωδικοποίηση του πεδίου Family_History ώστε όπου Yes = 1 και No =0

df["Radiation_Exposure"] = df["Radiation_Exposure"].map({"Yes": 1, "No": 0}) #Κωδικοποίηση του Radiation_Exposure ώστε όπου Yes = 1 και Νo =0

df["Iodine_Deficiency"] = df["Iodine_Deficiency"].map({"Yes": 1, "No": 0}) #Κωδικοποίηση του πεδίου Iodine_Deficiency με Yes = 1 και Νo =0

df["Smoking"] = df["Smoking"].map({"Yes": 1, "No": 0}) #Κωδικοποίηση του πεδίου Smoking με Yes = 1 και Νo =0

df["Obesity"] = df["Obesity"].map({"Yes": 1, "No": 0}) #Κωδικοποίηση του πεδίου Obesity με Yes = 1 και Νo =0

df["Diabetes"] = df["Diabetes"].map({"Yes": 1, "No": 0}) #Κωδικοποίηση του πεδίου Diabetes με Yes = 1 και Νo =0



#benign & malignant(καλοήθη & κακοήθη περιστατικά και όχι υγείή)

df["Diagnosis"] = df["Diagnosis"].map({"Malignant": 1, "Benign": 0}) #Κωδικοποίηση του πεδίου-στόχου Diagnosis.
                                      

df.to_excel("Dataset_Cleaned.xlsx",index=False) #Δημιουργία νέου excel με το καθαρισμένο dataset.
